# Лабораторная 17/2

Базовые методы обработки изображений.

*В этом ноутбуке изначально опущены результаты исполнения кода. Рекомендуется запускать (Shift+Enter) ячейки по мере просмотра документа*

Для начала подключим все библиотеки, которые нам понадобятся для выполнения заданий

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from imageio import imread, imsave
from skimage.color import rgb2gray

from skimage.filters import gaussian
from skimage.filters import median
from skimage.filters import sobel
from skimage.filters import prewitt
from skimage.filters import roberts
#from skimage.filters import laplace
from scipy.ndimage import laplace
from skimage.filters import meijering
from skimage.filters import sato
from skimage.filters import frangi
from skimage.filters import hessian

from skimage.filters import threshold_otsu
from skimage.filters import threshold_yen
from skimage.filters import threshold_isodata
from skimage.filters import threshold_li
from skimage.filters import threshold_local
from skimage.filters import threshold_minimum
from skimage.filters import threshold_triangle
from skimage.filters import try_all_threshold

from skimage.morphology import disk

## Задание 1. Матричные фильтры обработки изображений

Загрузим изображение с помощью библиотеки `imageio`:

In [ ]:
img = imread("bird.jpg")
plt.imshow(img)

### a) Фильтр Гаусса

Фильтр Гаусса (Gaussian filter) обычно используется в цифровом виде для обработки двумерных изображений с целью снижения уровня шума. Однако он дает сильное размытие изображения. Это иногда при анализе изображений искажает данные.

Как считается значение свертки для пикселя $(m,n)$: $$ y(m,n) = \frac {1}{2\pi r^2} \sum_{\substack{u, v}} e^{\frac{-(u^2+v^2)}{2r^2}} x(m+u, n+v) $$

$u$ принимает значения {-1,0,1};
$v$ принимает значения {-1,0,1};

Значение $r$ Вы можете подбирать самостоятельно для лучшего результата. 
Чтобы изменить значение $r$, в функции gaussian() необходимо изменять входной параметр $sigma$

In [ ]:
f, ax = plt.subplots(1, 3) # создадим сетку, в которую будем выводить картинки
f.set_dpi(100)             # повысим чёткость изображений

for i in range(3):
    ax[i].set_title("rgb"[i])                 # поставим подпись, соответствующую номеру канала
    ax[i].imshow(gaussian(img[...,i]))
    #ax[i].imshow(gaussian(img[..., i], sigma=2))

f.tight_layout() # добавим границы, чтобы графики не наезжали друг на друга

### b) Медианный фильтр

Медианный фильтр также применяется с целью избавления изображения от шума.

Как считается значение свертки медианным ядром:
* Значений у ядра нет
1. Ядро накладывается на участок изображения
2. Значение, входящие в эту область, сортируются в порядке возрастания/убывания
3. Находится медиана(Me) - это и будет результатом свертки

Медиана массива - это элемент, который стоит в середине отсортированного массива
Пусть есть массив чисел a. В массиве n элементов
Как считается медиана массива:
1. Если n(количество элементов) в массиве нечетное, то $$ Me = a[\frac{n+1}{2}] $$
2. Если n - четное, то $$ Me = \frac{a[\frac{n}{2}] + a[\frac{n}{2}+1]}{2} $$

In [ ]:
f, ax = plt.subplots(1, 3) 
f.set_dpi(100)         

for i in range(3):
    ax[i].set_title("rgb"[i])         
    ax[i].imshow(median(img[..., i], disk(1)))
    
f.tight_layout()

### c) Фильтр Собеля

Фильтр Собеля - это свертка с ядром 3х3, которое считается следующим образом:
1. Применяются 2 свертки к изображению последовательно, то есть вначале $G_{x}$, потом к этому же изображению применяется $G_{y}$ 
$$\\ \mathbf{G_{x}} = \left( \begin{array}{ccc} -1 & 0 & +1 \\ -2 & 0 & +2 \\ -1 & 0 & +1 \\ \end{array} \right) $$
 
$$\\ \mathbf{G_{y}} = \left( \begin{array}{ccc} -1 & -2 & -1 \\ 0 & 0 & 0 \\ +1 & +2 & +1 \\ \end{array} \right) $$

2. В итоге получаем 2 матрицы применения сверток. Затем необходимо посчитать $G = (G_{x}^2 + G_{y}^2)^{1/2}$ для каждогой пары элементов из матриц $G_{x}$, $G_{y}$


Фильтр Собеля применяется для нахождения границ на изображении.

In [ ]:
f, ax = plt.subplots(1, 3) 
f.set_dpi(100)           

for i in range(3):
    ax[i].set_title("rgb"[i])             
    ax[i].imshow(sobel(img[..., i]))
    
f.tight_layout()

### d) Фильтр Прюитта

Фильтр Прюитта - это свертка с ядром 3х3, которое считается следующим образом:
1. Применяются 2 свертки к изображению последовательно, то есть вначале $G_{x}$, потом к этому же изображению применяется $G_{y}$ 
$$\\ \mathbf{G_{x}} = \left( \begin{array}{ccc} -1 & 0 & +1\\ -1 & 0 & +1\\ -1 & 0 & +1\\ \end{array} \right)$$ 
 
$$ \mathbf{G_{y}} = \left( \begin{array}{ccc} -1 & -1 & -1\\ 0 & 0 & 0\\ +1 & +1 & +1\\ \end{array} \right) $$

2. В итоге получаем 2 матрицы применения сверток. Затем необходимо посчитать $G = (G_{x}^2 + G_{y}^2)^{1/2}$ для каждогой пары элементов из матриц $G_{x}$, $G_{y}$

Фильтр Прюитта применяется для нахождения границ на изображении.

In [ ]:
f, ax = plt.subplots(1, 3)
f.set_dpi(100)             

for i in range(3):
    ax[i].set_title("rgb"[i]) 
    ax[i].imshow(prewitt(img[..., i]))
    
f.tight_layout()

### e) Фильтр Робертса

Фильтр Робертса - это свертка с ядром 2х2, которое считается следующим образом:
1. Применяются 2 свертки к изображению последовательно, то есть вначале $G_{x}$, потом к этому же изображению применяется $G_{y}$ 
$$\\ \mathbf{G_{x}} = \left( \begin{array}{ccc} +1 & 0 \\ 0 & -1\\ \end{array} \right)$$ 
 
$$ \mathbf{G_{y}} = \left( \begin{array}{ccc} 0 & +1 \\ -1 & 0 \\ \end{array} \right) $$

2. В итоге получаем 2 матрицы применения сверток. Затем необходимо посчитать $G = (G_{x}^2 + G_{y}^2)^{1/2}$ для каждогой пары элементов из матриц $G_{x}$, $G_{y}$

Фильтр Робертса применяется для нахождения границ на изображении.

In [ ]:
f, ax = plt.subplots(1, 3)
f.set_dpi(100)      

for i in range(3):
    ax[i].set_title("rgb"[i])        
    ax[i].imshow(roberts(img[..., i]))
    
f.tight_layout()

In [ ]:
f, ax = plt.subplots(1, 3) # создадим сетку, в которую будем выводить картинки
f.set_dpi(100)             # повысим чёткость изображений

for i in range(3):
    ax[i].set_title("rgb"[i])                
    ax[i].imshow(meijering(img[..., i]))     
    
f.tight_layout() 

In [ ]:
f, ax = plt.subplots(1, 3) # создадим сетку, в которую будем выводить картинки
f.set_dpi(100)             # повысим чёткость изображений

for i in range(3):
    ax[i].set_title("rgb"[i])               
    ax[i].imshow(sato(img[..., i]))           
    
f.tight_layout() 

In [ ]:
f, ax = plt.subplots(1, 3) # создадим сетку, в которую будем выводить картинки
f.set_dpi(100)             # повысим чёткость изображений

for i in range(3):
    ax[i].set_title("rgb"[i])                
    ax[i].imshow(frangi(img[..., i]))       
    
f.tight_layout() 

## Задание 2. Бинаризация изображений

Бинаризация изображение - это процесс преобразования интенсивности пикселев в нули и единицы.

Бинаризация используется для поиска объекта на изображении и выделение конкректно его.

Сегментация - это деление пикселей изображения на пиксели, которые относятся к объекту(полезные), и пиксели, которые не относятся к объекту(фоновые). То есть значение "0" имеют "фоновые" пиксели, а "1"- "полезные" пиксели. 

In [ ]:
img_2 = imread('horse.jpg')
plt.imshow(img_2)

### Метод Оцу
Это алгоритм вычисления порога бинаризации

In [ ]:
f, ax = plt.subplots(1, 3)
f.set_dpi(100)

for i in range(3):
    ax[i].set_title("rgb"[i])
    img_gray = img_2[..., i]/255  # представление значения интенсивности пикселей в отрезке [0, 1]
    img_otsu = threshold_otsu(img_gray)  #               
    res_otsu = img_gray <= img_otsu
    ax[i].imshow(res_otsu, cmap='gray')
    
f.tight_layout()

Чтобы улучшить результат бинаризации, можно до применения метода Отцу использовать фильтры.

In [ ]:
f, ax = plt.subplots(1, 3)
f.set_dpi(100)

for i in range(3):
    ax[i].set_title("rgb"[i])
    img_conv = median(img_2[..., i])
    img_gray = img_conv/255  # представление значения интенсивности пикселей в отрезке [0, 1]
    img_otsu = threshold_otsu(img_gray)  #               
    res_otsu = img_gray <= img_otsu
    ax[i].imshow(res_otsu, cmap='gray')
    
f.tight_layout()